# 1) Large size vectors: lcc MAD-Mex

## a) Normalize, reproject to wgs84

In [ ]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-issue-3/python3_package_for_geonode/geonode_conabio'

In [4]:
os.chdir(dir_alg_python)

In [5]:
import fiona
from pyproj import Proj
from fiona.crs import to_string

from wrappers import reproj_normalize_and_write_large_vector


In [7]:
#direc = "/LUSTRE/MADMEX/products/landcoverchange/landsat/mosaics/2017_2018"
direc = "/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/HIDALGO/"

In [8]:
#input_filename = direc + "/madmex_landsat_changes_2017-2018.shp"
#output_filename = direc + "/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp"
#layer = "madmex_landsat_changes_2017-2018_wgs84_using_fiona"
input_filename = direc + "HIDALGO_merge.shp"
output_filename = direc + "HIDALGO_merge_wgs84_using_fiona.shp"
layer = "HIDALGO_merge_wgs84_using_fiona"
#dst_crs = "4326"
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [9]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

**Read and call wrapper**

In [13]:
with fiona.open(input_filename) as src:
    fc = (feature for feature in src if not None in [feature['properties'][att] for att in list_name_attributes] and
                                                     feature['geometry'] is not None)
    src_crs = to_string(src.crs)
    proj_src_crs = Proj(src_crs)
    fc_schema = src.schema
    if not proj_src_crs.crs.is_geographic:
        reproj_normalize_and_write_large_vector(fc, fc_schema,
                                                list_name_attributes,
                                                layer,
                                                output_filename,
                                                src_crs, 
                                                is_geographic = False
                                                )
    else:
        reproj_normalize_and_write_large_vector(fc, fc_schema, 
                                                list_name_attributes,
                                                layer,
                                                output_filename,
                                                src_crs
                                                )
        

## b) Register large vector layer in geonode via `shp2pgsql` and `updatelayers`

**Make sure stack deployment of geonode is up and running**

**1) Create `~/.pgpass` with contents: `hostname:port:database:username:password` and `chmod 0600 ~/.pgpass`**

In [9]:
from utils_cli import create_table_via_shp2pgsql

In [10]:
#next need to be in lowercase
name_table = "hidalgo_madmex_lcc_2017_2018_fiona" #madmex_landsat_2017-2018_lcc


In [24]:
[result_create_table, name_table, out, err] = create_table_via_shp2pgsql(output_filename, name_table)

hidalgo_madmex_lcc_2017_2018_fiona
after wait


In [25]:
print((result_create_table, out, err))

(0, None, b'')


**2) Add to geoserver from geonode_data database**. Need to follow: https://training.geonode.geo-solutions.it/004_admin_workshop/007_loading_data_into_geonode/geoserver.html

In [1]:
from docker import APIClient


In [2]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [3]:
string1 = "from geoserver.catalog import Catalog;"


In [4]:
string2 = "cat = Catalog('http://geonode.conabio.gob.mx' + '/geoserver/rest', \'"

In [5]:
user = "miuser"

In [6]:
passwd = "mipassword"

In [8]:
string3 = "gstore = cat.get_stores(names=['geonode_data'], workspaces=['geonode']);"

In [9]:
epsg_code = "EPSG:4326"

In [10]:
native_name = "aguascalientes_merge_wgs84_fiona"

In [11]:
string4 = "cat.publish_featuretype(native_name, gstore[0], epsg_code, native_name=native_name)"

In [12]:
string = "".join([string, string2, 
                  user, "\', ", "\'", passwd, "\');",
                  string3,
                  "native_name = \'", native_name, "\';", 
                  "epsg_code = \'", epsg_code, "\';", 
                  string4])

In [14]:
cmd = "".join(["python -c ", "\"", string, "\""])

In [16]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)


In [17]:
ex_start = c.exec_start(exec_id=ex)

In [18]:
print(ex_start)

b'Traceback (most recent call last):\n  File "<string>", line 1, in <module>\n  File "/usr/local/lib/python3.7/site-packages/geoserver/catalog.py", line 889, in publish_featuretype\n    raise FailedRequestError(\'Failed to publish feature type {} : {}, {}\'.format(name, resp.status_code, resp.text))\ngeoserver.catalog.FailedRequestError: Failed to publish feature type aguascalientes_merge_wgs84_fiona : 500, Resource named \'aguascalientes_merge_wgs84_fiona\' already exists in store: \'geonode_data\'\n'


**3) Update layers using docker container `spcgeonode_django_1`**

In [11]:
from utils_docker import update_layers_via_docker

In [12]:
result_update = update_layers_via_docker(name_table)

In [14]:
#print(result_update)

**4) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# 2) Small or medium size raster: lc MAD-Mex

## a) Reproject to wgs84, compress and tile it

Based on: 

https://rasterio.readthedocs.io/en/latest/topics/reproject.html

https://rasterio.readthedocs.io/en/latest/topics/image_options.html

In [1]:
from pyproj import Proj
import rasterio

from geonode_conabio.wrappers import reproj_and_write_one_band_raster

In [2]:
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Aguascalientes/31/'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31/'
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [4]:
#input_filename = ''.join([direc, '/madmex_sentinel2_2017_31.tif'])
#output_filename = ''.join([direc, '/madmex_sentinel2_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Aguascalientes_2017_31.tif'])
#output_filename = ''.join([direc, '/Aguascalientes_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Hidalgo_2017_31.tif'])
#output_filename = ''.join([direc, '/Hidalgo_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = "".join([direc, "/eoss4tnc_landcover_landsat8_2016_20200220_auxlabel_madmexplus_QRO.tif"])
#output_filename = "".join([direc, "/eoss4tnc_landcover_landsat8_2016_20200220_auxlabel_madmexplus_QRO_rasterio.tif"])
input_filename = "".join([direc, "madmex_lc_2015.tif"])
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])
#dst_crs = 'EPSG:4326'

In [5]:
with rasterio.open(input_filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        reproj_and_write_one_band_raster(src, output_filename,
                                         is_geographic=False)
    else:
        reproj_and_write_one_band_raster(src, output_filename)

## b) Register small, medium or large raster layer in geonode via `importlayers`

**1) Importlayers cmd**

In [1]:
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [2]:
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])

In [3]:
region = "Mexico, North America, Latin America"
name="National_MAD-Mex_landsat8_lc_2015_32_classes"
title="National land cover map 2015 32 classes"
kw="MAD-Mex, Landsat8, GeoTIFF, WCS"
abstract="Land cover sourced by Landsat8 satellite images. All available images for the required calendar years were acquired through the ESPA processing system and were provided as atmospherically corrected surface reflectance and are accompanied by quality layers assigning, amongst others, pixels obscured by clouds and cloud shadows. Land cover is generated using a novel semi automated workflow based on time series processing and optimized training data generation. The workflow consists of the following procedures, including data acquisition, time series generation and processing, spectral clustering, training data generation and optimization, supervised object-based classifications, and geometry harmonization. "


In [4]:
region = ''.join(["\"", region, "\""])
name = ''.join(["\"", name, "\""])
title = ''.join(["\"", title, "\""])
abstract = ''.join(["\"", abstract, "\""])
kw = ''.join(["\"", kw, "\""])

In [5]:
from geonode_conabio.utils_docker import import_layers_via_docker

/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-master/python3_package_for_geonode/geonode_conabio/settings.py:12: UserWarning: No configuration file found in /root/.geonode_conabio, some functionalities won't be available
  warnings.warn('No configuration file found in %s, some functionalities won\'t be available' % dotenv_path)


In [6]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [14]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# 2) Large size raster: lc MAD-Mex

## a) Reproject to wgs84, compress and tile it

In [1]:
from pyproj import Proj
import rasterio

from geonode_conabio.wrappers import reproj_and_write_one_band_raster

In [2]:
direc = "/LUSTRE/MADMEX/tnc_data_steffen_thilo/tnc_marismas/"

In [3]:
input_filename = "".join([direc, "marismasnacionales_coberturadesuelo_2019_nivel2_final_20190925_relabeled.tif"])
output_filename = "".join([direc, "MAD-Mex_TNC_marismas_nacionales_landsat8_rapideye_lc_2018_2019_13_classes.geotiff"])
#dst_crs = 'EPSG:4326'

In [4]:
output_filename

'/LUSTRE/MADMEX/tnc_data_steffen_thilo/tnc_marismas/MAD-Mex_TNC_marismas_nacionales_landsat8_rapideye_lc_2018_2019_13_classes.geotiff'

In [5]:
with rasterio.open(input_filename) as src:   
    reproj_and_write_one_band_raster(src, output_filename)

## b) Register large raster layer in geonode via `importlayers`

**1) Importlayers cmd**

In [1]:
direc = '/LUSTRE/MADMEX/products/landcover/landsat/2015/32/'

In [2]:
output_filename = "".join([direc, "madmex_lc_2015_wgs84_tiled_rasterio.tif"])

In [3]:
region = "Mexico, North America, Latin America"
name="National_MAD-Mex_landsat8_lc_2015_32_classes"
title="National land cover map 2015 32 classes"
kw="MAD-Mex, Landsat8, GeoTIFF, WCS"
abstract="Land cover sourced by Landsat8 satellite images. All available images for the required calendar years were acquired through the ESPA processing system and were provided as atmospherically corrected surface reflectance and are accompanied by quality layers assigning, amongst others, pixels obscured by clouds and cloud shadows. Land cover is generated using a novel semi automated workflow based on time series processing and optimized training data generation. The workflow consists of the following procedures, including data acquisition, time series generation and processing, spectral clustering, training data generation and optimization, supervised object-based classifications, and geometry harmonization. "


In [4]:
region = ''.join(["\"", region, "\""])
name = ''.join(["\"", name, "\""])
title = ''.join(["\"", title, "\""])
abstract = ''.join(["\"", abstract, "\""])
kw = ''.join(["\"", kw, "\""])

In [5]:
from geonode_conabio.utils_docker import import_layers_via_docker

/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-master/python3_package_for_geonode/geonode_conabio/settings.py:12: UserWarning: No configuration file found in /root/.geonode_conabio, some functionalities won't be available
  warnings.warn('No configuration file found in %s, some functionalities won\'t be available' % dotenv_path)


In [6]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [14]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

**3) Pack tif and style into zip and copy to path for ftp**

**Read sld and write cmap to raster**

References:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/forests_maps_julian/model_predict_and_db_to_raster_pipeline.ipynb

https://github.com/CONABIO/antares3/blob/develop/madmex/util/db.py

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/rapideye_parsing_metadaata/metadata_parser.ipynb

References for zipfile:

https://thispointer.com/python-how-to-create-a-zip-archive-from-multiple-files-or-directory/

https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory-in-python

```
/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/madmex_32_classes.sld

/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/madmex_31_classes.sld

/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/data/geonode/Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512/Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512.geotiff

```

In [1]:
title_layer = "My title" #this will be input of function
destiny_path = "/data/var/ftp/pub/" #input of function
dir_path_styles_geonode = "/LUSTRE/MADMEX/geonode_spc_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/" #input of function
dir_path_layers_geonode = "/LUSTRE/MADMEX/geonode_spc_volume/geonode/scripts/spcgeonode/_volume_geodatadir/data/geonode/" #input of function

In [2]:
from geonode_conabio.utils_docker import get_layer_and_style_registered_in_geonode_via_docker
from geonode_conabio.utils_style import get_cmap_from_element_list_madmex_style
from geonode_conabio.utils_parser import parse_sld_madmex_style

In [3]:
result_get = get_layer_and_style_registered_in_geonode_via_docker(title_layer)

In [4]:
print(result_get)

b'National_MAD-Mex_landsat8_lc_2015_32_classes\nmadmex_landcover_32_classes_raster\n'


In [5]:
result_get.decode().split("\n")[0:2]

['Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512',
 'madmex_32_classes']

In [6]:
layer_name, style_name = result_get.decode().split("\n")[0:2]

In [7]:
style_source_path =  dir_path_styles_geonode + style_name + ".sld"
layer_source_path =  dir_path_layers_geonode + layer_name + "/" + layer_name + ".geotiff"

In [8]:
rf_children = parse_sld_madmex_style(style_source_path)

In [9]:
rgb_dict = get_cmap_from_element_list_madmex_style(rf_children)

In [10]:
import os
import rasterio


Write cmap to raster

In [11]:
layer_cmap_source_path = destiny_path + layer_name + ".geotiff"

In [12]:
with rasterio.open(layer_source_path) as src:
    arr = src.read(1)
    meta = src.meta


In [13]:
meta["compress"] = "lzw"

In [14]:
with rasterio.open(layer_cmap_source_path, 'w', **meta) as dst:
    dst.write(arr, indexes=1)
    dst.write_colormap(1,rgb_dict)
    cmap_result = dst.colormap(1)

In [15]:
import zipfile
from os.path import basename

In [16]:
zip_file = destiny_path + layer_name + ".zip"

In [17]:
zip_file

'/shared_volume/ftp_dir/Test3_National_MAD-Mex_landsat8_lc_2015_32_classes_512_512.zip'

In [18]:
zipf = zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED)
zipf.write(style_source_path, basename(style_source_path))
zipf.write(layer_cmap_source_path, basename(layer_cmap_source_path))
zipf.close()

In [19]:
os.remove(layer_cmap_source_path)

**4) Execute update link for original dataset**

References:


https://github.com/GeoNode/geonode/blob/master/geonode/utils.py#L1668

https://github.com/GeoNode/geonode/blob/89ea8460f23b90c085b33b10b8f0700b9ab506d4/geonode/base/management/commands/set_all_layers_metadata.py#L116

https://github.com/CONABIO/geonode/blob/master/notebooks/3_geonode_metadata_update.ipynb



In `/shared_volume/geonode/geonode/settings.py` remove `GZIP` as its not working. First stop and then up:

```
docker-compose stop

docker-compose up -d django geoserver postgres nginx
```

And remove `GZIP` from `/shared_volume/geonode/geonode/settings.py`

```
DOWNLOAD_FORMATS_RASTER = [
    'JPEG',
    'PDF',
    'PNG',
    'ArcGrid',
    'GeoTIFF',
    'Gtopo30',
    'ImageMosaic',
    'KML',
    'View in Google Earth',
    'Tiles',
    'GML',
    'QGIS layer file (.qlr)',
    'QGIS project file (.qgs)',
    'Zipped All Files'
]
```

```
#being in docker exec -it spcgeonode_django_1 bash

import os
import django

os.chdir("/spcgeonode")

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "geonode.local_settings")

django.setup()

from geonode.base.models import Link

from geonode.layers.models import Layer

layer = Layer.objects.filter(title="My title").first()

print(layer.title)

download_url = "the url"

extension = "zip"

name = "Original Dataset" #could be "Raw Data"

mime = "application/octet-stream"

link_type = "original"

Link.objects.update_or_create(resource=layer, url=download_url, defaults=dict(extension=extension, name=name, mime=mime, link_type=link_type))


```


```
#to delete link:

Link.objects.filter(resource=layer,url=download_url).first().delete()

```

In [20]:
from geonode_conabio.utils_docker import create_link_in_geonode_for_zip_file_via_docker

In [21]:
create_link_in_geonode_for_zip_file_via_docker(zip_file, title_layer)

b''

# 3) Small to Medium size vectors: lcc MAD-Mex


## a) Normalize, reproject to wgs84

In [1]:
import os

import geopandas as gpd

from geonode_conabio.wrappers import reproj_normalize_and_write_small_medium_size_vector

In [2]:
direc_input_filename = "/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/"
direc_output_filename = "/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1_feb_2020_normalized_attributes/"

In [3]:
name_output = "cultivos2019_normalized_string_attributes"

In [4]:
input_file = os.path.join(direc_input_filename, "cultivos2019.shp")
output_filename = os.path.join(direc_output_filename, name_output)
layer = name_output

**Read and call wrapper**

In [5]:
gdf = gpd.read_file(input_file)


In [6]:
gdf

,PuntoID,Fecha,Municipio,DistritoDR,Ecoregion,ZonadeVida,Cultivo,Asociació,X,Y,Observacio,geometry
0,1,mar-19,Catemaco,San Andres Tuxtla,Sierra de los Tuxtlas,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Café,NA,-95.105416,18.341956,CERCA PLÁTANO,POINT (-95.10542 18.34196)
1,2,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Hule hevea,NA,-94.098841,17.864591,None,POINT (-94.09884 17.86459)
2,3,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.100192,17.867630,None,POINT (-94.10019 17.86763)
3,8,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Maíz,NA,-94.110911,17.814089,None,POINT (-94.11091 17.81409)
4,9,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.106537,17.765451,None,POINT (-94.10654 17.76545)
...,...,...,...,...,...,...,...,...,...,...,...,...
14633,15178,dic-19,Agua Dulce,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.172940,18.170492,None,POINT (-94.17294 18.17049)
14634,15179,dic-19,Agua Dulce,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.106903,18.019101,None,POINT (-94.10690 18.01910)
14635,15180,dic-19,Uxpanapa,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso montano bajo [Cálido - Muy Llu...,Hule hevea,NA,-94.131075,17.238747,None,POINT (-94.13107 17.23875)
14636,15182,dic-16,Uxpanapa,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso montano bajo [Cálido - Muy Llu...,Pastizal,NA,-94.107095,17.228911,None,POINT (-94.10709 17.22891)


In [7]:
gdf.rename(columns = {"Asociació": "Asociacion",
                      "Observacio": "Observ"}, inplace=True)

In [8]:
gdf

,PuntoID,Fecha,Municipio,DistritoDR,Ecoregion,ZonadeVida,Cultivo,Asociacion,X,Y,Observ,geometry
0,1,mar-19,Catemaco,San Andres Tuxtla,Sierra de los Tuxtlas,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Café,NA,-95.105416,18.341956,CERCA PLÁTANO,POINT (-95.10542 18.34196)
1,2,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Hule hevea,NA,-94.098841,17.864591,None,POINT (-94.09884 17.86459)
2,3,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.100192,17.867630,None,POINT (-94.10019 17.86763)
3,8,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Maíz,NA,-94.110911,17.814089,None,POINT (-94.11091 17.81409)
4,9,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.106537,17.765451,None,POINT (-94.10654 17.76545)
...,...,...,...,...,...,...,...,...,...,...,...,...
14633,15178,dic-19,Agua Dulce,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.172940,18.170492,None,POINT (-94.17294 18.17049)
14634,15179,dic-19,Agua Dulce,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.106903,18.019101,None,POINT (-94.10690 18.01910)
14635,15180,dic-19,Uxpanapa,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso montano bajo [Cálido - Muy Llu...,Hule hevea,NA,-94.131075,17.238747,None,POINT (-94.13107 17.23875)
14636,15182,dic-16,Uxpanapa,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso montano bajo [Cálido - Muy Llu...,Pastizal,NA,-94.107095,17.228911,None,POINT (-94.10709 17.22891)


In [18]:
name_attribute1 = "Fecha"
name_attribute2 = "Municipio"
name_attribute3 = "DistritoDR"
name_attribute4 = "Ecoregion"
name_attribute5 = "ZonadeVida"
name_attribute6 = "Cultivo"
name_attribute7 = "Asociacion"
name_attribute8 = "Observ"

In [19]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6,
                        name_attribute7, name_attribute8
                       ]

name_attribute2, name_attribute4, name_attribute5, name_attribute7

In [20]:
#gdf.loc[gdf[[name_attribute2]].isna().any(1)].index

In [21]:
#gdf.iloc[11950]

In [22]:
#gdf.dropna(subset=[name_attribute7], inplace=True)

In [23]:
#gdf

In [24]:
#gdf.dropna(subset=["Observacio"], inplace=True)

In [25]:
gdf.dropna(subset=list_name_attributes, inplace=True, how="all")

In [26]:
gdf

,PuntoID,Fecha,Municipio,DistritoDR,Ecoregion,ZonadeVida,Cultivo,Asociacion,X,Y,Observ,geometry
0,1,mar-19,Catemaco,San Andres Tuxtla,Sierra de los Tuxtlas,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Café,NA,-95.105416,18.341956,CERCA PLÁTANO,POINT (-95.10542 18.34196)
1,2,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Hule hevea,NA,-94.098841,17.864591,None,POINT (-94.09884 17.86459)
2,3,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.100192,17.867630,None,POINT (-94.10019 17.86763)
3,8,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Maíz,NA,-94.110911,17.814089,None,POINT (-94.11091 17.81409)
4,9,mar-19,Las Choapas,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.106537,17.765451,None,POINT (-94.10654 17.76545)
...,...,...,...,...,...,...,...,...,...,...,...,...
14633,15178,dic-19,Agua Dulce,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.172940,18.170492,None,POINT (-94.17294 18.17049)
14634,15179,dic-19,Agua Dulce,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,NA,-94.106903,18.019101,None,POINT (-94.10690 18.01910)
14635,15180,dic-19,Uxpanapa,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso montano bajo [Cálido - Muy Llu...,Hule hevea,NA,-94.131075,17.238747,None,POINT (-94.13107 17.23875)
14636,15182,dic-16,Uxpanapa,Choapas,Petén-Veracruz moist forests,Bosque Lluvioso montano bajo [Cálido - Muy Llu...,Pastizal,NA,-94.107095,17.228911,None,POINT (-94.10709 17.22891)


In [27]:
input_file

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/cultivos2019.shp'

In [28]:
output_filename

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1_feb_2020_normalized_attributes/cultivos2019_normalized_string_attributes'

In [29]:
output_filename_geonode = reproj_normalize_and_write_small_medium_size_vector(gdf, 
                                                                              list_name_attributes,
                                                                              layer,
                                                                              direc_input_filename,
                                                                              output_filename)



DriverError: sqlite3_open(/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1_feb_2020_normalized_attributes/cultivos2019_normalized_string_attributes.gpkg) failed: unable to open database file

In [12]:
print(output_filename_geonode)

/shared_volume/ftp_dir/test3_samof_vector/test3_samof_vector.shp


In [20]:
asociacio = gdf["Asociació"]

In [21]:
asociacio

0        NA
1        NA
2        NA
3        NA
4        NA
         ..
14633    NA
14634    NA
14635    NA
14636    NA
14637    NA
Name: Asociació, Length: 14638, dtype: object

In [27]:
asociacio.unique()

array(['NA', 'Café Sol-Limón', 'Cítrico-Café', 'Cítrico-Café Sol',
       'Cítrico-Maíz', 'Cítrico-Plátano', 'Café-Cítrico'], dtype=object)

In [28]:
asociacio.value_counts()

NA                  14622
Cítrico-Maíz            7
Cítrico-Plátano         4
Cítrico-Café Sol        2
Café Sol-Limón          1
Café-Cítrico            1
Cítrico-Café            1
Name: Asociació, dtype: int64

## b) Register vector layer in geonode via `importlayers`

**1) Importlayers cmd**

In [13]:
region = "".join(["\"Mexico, North America, Latin America\""])
#name = "\"samof_madmex_vector\""
name_geonode = ''.join(['\"', name, '\"'])
title = "\"Test3 Samof MAD-MEX\""
abstract = "\"MAD-Mex land cover change\""
kw = "\"MAD-Mex, WCS\""


In [14]:
name_geonode

'"test3_samof_vector"'

In [15]:
from docker import APIClient


In [16]:
c = APIClient(base_url='tcp://172.17.0.1:1111')

In [17]:
cmd = "".join(["python manage.py importlayers -v 3 -i -o ",
               "-n ", name_geonode, " ",
               "-t ", title, " ",
               "-a ", abstract, " ",
               "-k ", kw, " ",
               "-r ", region, " ",
               output_filename_geonode, " ",
               "--settings=geonode.local_settings"])
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)


In [18]:
ex_start = c.exec_start(exec_id=ex)

In [23]:
#print(ex_start)

In [19]:
output_filename_without_extension = output_filename_geonode.split('.')[0]

In [20]:
from glob import glob

In [21]:
l_output_filenames = glob(output_filename_without_extension +'*',recursive=True)

In [22]:
l_output_filenames

['/shared_volume/ftp_dir/test3_samof_vector/test3_samof_vector.dbf',
 '/shared_volume/ftp_dir/test3_samof_vector/test3_samof_vector.prj',
 '/shared_volume/ftp_dir/test3_samof_vector/test3_samof_vector.shx',
 '/shared_volume/ftp_dir/test3_samof_vector/test3_samof_vector.shp',
 '/shared_volume/ftp_dir/test3_samof_vector/test3_samof_vector.cpg']

In [23]:
for filenames in l_output_filenames:
    os.remove(filenames)

In [24]:
os.removedirs(output_filename)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

**3) Create style in geoserver manually**

# c) Pack gpkg, style and copy to path for ftp

In [1]:
title_layer = "Test3 Samof MAD-MEX" #this will be input of function
destiny_path = "/shared_volume/ftp_dir/" #input of function
dir_path_styles_geonode = "/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/" #input of function
#dir_path_layers_geonode = "/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/data/geonode/" #input of function

dir_path_layers_geonode = destiny_path

In [2]:
from geonode_conabio.utils_docker import get_layer_and_style_registered_in_geonode_via_docker
from geonode_conabio.utils_style import get_cmap_from_element_list_madmex_style
from geonode_conabio.utils_parser import parse_sld_madmex_style

/shared_volume/geonode_conabio/python3_package_for_geonode/geonode_conabio/settings.py:12: UserWarning: No configuration file found in /root/.geonode_conabio, some functionalities won't be available
  warnings.warn('No configuration file found in %s, some functionalities won\'t be available' % dotenv_path)


In [3]:
result_get = get_layer_and_style_registered_in_geonode_via_docker(title_layer)

In [4]:
print(result_get)

b'test3_samof_vector\ntest3_samof_vector\n'


In [5]:
result_get.decode().split("\n")[0:2]

['test3_samof_vector', 'test3_samof_vector']

In [6]:
layer_name, style_name = result_get.decode().split("\n")[0:2]

In [7]:
layer_name

'test3_samof_vector'

In [8]:
style_name

'test3_samof_vector'

In [9]:
import os

In [10]:
style_source_path =  os.path.join(dir_path_styles_geonode, style_name +  ".sld")
layer_source_path =  os.path.join(dir_path_layers_geonode, layer_name + ".gpkg")

In [11]:
layer_source_path

'/shared_volume/ftp_dir/test3_samof_vector.gpkg'

In [12]:
style_source_path

'/shared_volume/geonode/scripts/spcgeonode/_volume_geodatadir/workspaces/geonode/styles/test3_samof_vector.sld'

In [13]:
from geonode_conabio.utils_os import zip_dir_for_layer_and_style

In [14]:
#Zip dir        
zip_file = os.path.join(destiny_path, layer_name)
zip_file = ''.join([zip_file, ".zip"])



In [15]:
zip_file

'/shared_volume/ftp_dir/test3_samof_vector.zip'

In [25]:
zip_dir_for_layer_and_style(zip_file, layer_source_path, style_source_path) 

In [16]:
#if success creating download link in geonode
print(os.path.basename(zip_file))


test3_samof_vector.zip


In [17]:
download_path = "miftp/"

In [18]:
download_url = os.path.join(download_path, os.path.basename(zip_file))


In [19]:
download_url

'miftp/test3_samof_vector.zip'

In [21]:
from geonode_conabio.utils_docker import create_link_in_geonode_for_zip_file_via_docker

In [22]:
if os.path.exists(zip_file):
    create_link_in_geonode_for_zip_file_via_docker(download_url, title_layer)  

# Towards python3 package

Following

https://python-packaging.readthedocs.io/en/latest/minimal.html